<a href="https://colab.research.google.com/github/kashifimteyaza/LLM-Mixers/blob/main/CoT_Adap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" --q

import torch
from unsloth import FastLanguageModel

# Check CUDA compatibility
major_version, minor_version = torch.cuda.get_device_capability()

# Instantiate FastLanguageModel
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="google/gemma-2b-it",
    max_seq_length=4096,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 M

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:123: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Gemma patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.22.post7. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.07G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


**Using Partial Embedding Fine-Tuning**- It selectively fine-tunes a limited number of additional model parameters while keeping the majority of pretrained LLM parameters frozen!

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2024.10.7 patched 18 layers with 18 QKV layers, 18 O layers and 18 MLP layers.


In [6]:
# Check the dataset's column names
print(ds.column_names)


['source', 'target', 'rationale', 'task', 'type']


In [7]:
from datasets import load_dataset

# Define the prompt template
prompt = """Based on given instruction and context, generate an appropriate response

### Instruction:
{}

### Context:
{}

### Response:
{}
"""

# Define the EOS token and formatting function
EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    # Assuming 'task' is the instruction, 'source' as context, and 'target' as response
    instructions = examples["task"]
    contexts = examples["source"]
    responses = examples["target"]
    texts = []

    for i, j, k in zip(instructions, contexts, responses):
        text = prompt.format(i, j, k) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

# Load and process the dataset
ds = load_dataset("Imteyazk/CoT-filtered", split="train")
dataset = ds.map(formatting_prompts_func, batched=True)  # Use ds.map instead of dataset.map


Map:   0%|          | 0/400404 [00:00<?, ? examples/s]

In [8]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 2,
        max_steps = 10,
        learning_rate = 0.0005,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Training
trainer_stats = trainer.train()

Map (num_proc=2):   0%|          | 0/400404 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 400,404 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 19,611,648


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir --no-deps unsloth transformers git+https://github.com/huggingface/trl.git`


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
1,2.922000
2,2.981500
3,3.010100
4,2.629900
5,2.779600
6,2.546800
7,2.433600
8,2.537300
9,2.398000
10,2.134800


In [11]:
from huggingface_hub import login

# Log in with your Hugging Face token
login("Enter_your_Token")


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [21]:
from transformers import AutoTokenizer
from unsloth import FastLanguageModel
from transformers import TextStreamer

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model, _ = FastLanguageModel.from_pretrained("google/gemma-2b-it")  # Load FastLanguageModel
model = FastLanguageModel.for_inference(model)  # Set model for inference

# Define student profile for personalization
student_profile = {
    "name": "Kwame Mensah",
    "age": 21,
    "major": "Political Science",
    "interests": "African politics and development economics",
    "background": "Accra",
    "language": "English"
}

# Define a detailed and explicitly structured prompt for each law
prompt_template = """
Generate a clear, structured, and detailed explanation of Newton's three laws of motion for {name}, a {age}-year-old {major} major interested in {interests}. Explain each law and relate it to political and economic principles that would be meaningful to {name}. Structure the response as follows:

1. First Law of Motion:
   * This law states that an object at rest will remain at rest, or if moving, will continue in a straight line at a constant speed, unless acted upon by an external force.
2. Second Law of Motion:
   * This law determines the relationship between the force applied to an object and its resulting acceleration.

3. Third Law of Motion:
   * This law states that for every action, there is an equal and opposite reaction.
"""

# Create input prompt using student profile
text = prompt_template.format(**student_profile)

# Tokenize the input
inputs = tokenizer(text, return_tensors="pt")

# Create a text streamer for output
text_streamer = TextStreamer(tokenizer)

# Generate structured and personalized response
output = model.generate(
    inputs['input_ids'],
    streamer=text_streamer,
    max_new_tokens=512,
    top_k=50,
    top_p=0.9,
    temperature=0.7,
    no_repeat_ngram_size=3
)

# Decode and print the generated response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


==((====))==  Unsloth 2024.10.7: Fast Gemma patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.22.post7. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
<bos>
Generate a clear, structured, and detailed explanation of Newton's three laws of motion for Kwame Mensah, a 21-year-old Political Science major interested in African politics and development economics. Explain each law and relate it to political and economic principles that would be meaningful to Kwame Mensah. Structure the response as follows:

1. First Law of Motion:
   * This law states that an object at rest will remain at rest, or if moving, will continue in a straight line at a constant speed, unless acted upon by an external force. 
2